# Yfinance, Option Chain, and Data

Focus: Gain familiarity with Yfinance, the option chain data, and how to manipulate the data for later computations of implied volatilty surfaces and PCA.

Notes:
- The ability to calculate an implied volatity through BS, simple samples, and rootfinding methods has already been implemented at this point 

- The focus of this notebook will be to further these implementations with real market data

- There will be conceptual note for both myself and other to follow along in this notebook

In [2]:
#Allow imports from the src directory
import sys
from pathlib import Path
project_root = Path().resolve().parents[0]
sys.path.append(str(project_root))

# Import self-made libraries to run checks
from src.black_scholes import black_scholes_price
from src.implied_vol import implied_volatility

# Standard imports
import yfinance as yf
import pandas as pd
import numpy as np

Cell 1 Focus/Notes:
.option_chain()
- Have to pass in a date of string type labeled as "YYYY-MM-DD"
- Cannot pull multiple dates without a loop and adding to data frame

In [9]:
#Initialization 
ticker = "SPY"
tk = yf.Ticker(ticker)

#Example of not using a specific expiration date
SPYdf = tk.option_chain()
print("SPYdf data type:", type(SPYdf), "<- Notes that it does not return a DataFrame. Instead: <class 'tuple'>\n")

#Pulling available expiration dates
expirations = tk.options
print("SPY Expirations:", expirations, "\n")

#Pull option chain for a specific date
SPYdf = tk.option_chain('2026-02-20')
print("SPYdf data type:", type(SPYdf), "<- Now its a pndas DataFrame\n")
print("View of dataframe:\n")
print(SPYdf)



SPYdf data type: <class 'yfinance.ticker.Options'> <- Notes that it does not return a DataFrame. Instead: <class 'tuple'>

SPY Expirations: ('2026-02-06', '2026-02-09', '2026-02-10', '2026-02-11', '2026-02-12', '2026-02-13', '2026-02-20', '2026-02-27', '2026-03-06', '2026-03-13', '2026-03-20', '2026-03-31', '2026-04-17', '2026-04-30', '2026-05-15', '2026-05-29', '2026-06-18', '2026-06-30', '2026-07-31', '2026-09-18', '2026-09-30', '2026-12-18', '2026-12-31', '2027-01-15', '2027-03-19', '2027-06-17', '2027-12-17', '2028-01-21', '2028-06-16', '2028-12-15') 

SPYdf data type: <class 'yfinance.ticker.Options'> <- Now its a pndas DataFrame

View of dataframe:

Options(calls=         contractSymbol             lastTradeDate  strike  lastPrice     bid  \
0    SPY260220C00335000 2026-02-02 19:44:19+00:00   335.0     362.17  351.79   
1    SPY260220C00345000 2026-02-02 14:42:12+00:00   345.0     348.01  341.78   
2    SPY260220C00350000 2026-02-03 19:25:15+00:00   350.0     337.41  336.89   
3 